# INFO 2950 Group Project: What Makes a Nobel Prize Laureate?

## Research Questions:
1. What region of the world produces the most Nobel Prize laureates?
2. At what age is your likelihood of winning the highest?
3. Does age at the time of winning the Nobel Prize vary between prize categories?
3. Is there a relationship between being affliated with a higher-ranked university and winning the Nobel Prize?
    a. Which universities have dominance in each of the prize categories?
4. Does Nobel Prize favor one gender over another?
5. What proportion of laureates won the Nobel Prize twice or more?
6. What proportion of laureates are family members?

## Data & Data Cleaning

In [1]:
import pandas as pd 
import numpy as np
import seaborn 
from matplotlib import pyplot

In [4]:
nobel_data_raw = pd.read_csv("laureate.csv")
print(nobel_data_raw.head())

   id       firstname    surname        born        died  \
0   1  Wilhelm Conrad    Röntgen  1845-03-27  1923-02-10   
1   2  Hendrik Antoon    Lorentz  1853-07-18  1928-02-04   
2   3          Pieter     Zeeman  1865-05-25  1943-10-09   
3   4   Antoine Henri  Becquerel  1852-12-15  1908-08-25   
4   5          Pierre      Curie  1859-05-15  1906-04-19   

             bornCountry bornCountryCode                bornCity  \
0  Prussia (now Germany)              DE  Lennep (now Remscheid)   
1        the Netherlands              NL                  Arnhem   
2        the Netherlands              NL              Zonnemaire   
3                 France              FR                   Paris   
4                 France              FR                   Paris   

       diedCountry diedCountryCode   diedCity gender    year category  \
0          Germany              DE     Munich   male  1901.0  physics   
1  the Netherlands              NL        NaN   male  1902.0  physics   
2  the Neth

## Data Description 

## Data Limitations

## Exploratory Data Analysis


## Questions For Reviewers


- Since our data is collected from the Harvard Dataverse site in 2016, we are missing data from 2017 to 2022. Should we keep the dataset as is or add the missing ones manually? 